In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def classifier(input_shape, width, kernel_size, pool_size):
    model = Sequential()

    model.add(Convolution3D(width*1, kernel_size[0], kernel_size[1], kernel_size[2],
                            border_mode='valid',
                            input_shape=input_shape))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    #model.add(Dropout(0.2))
    model.add(Convolution3D(width*2, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    #model.add(Dropout(0.2))
    model.add(Convolution3D(width*4, kernel_size[0], kernel_size[1], kernel_size[2]))
    model.add(Activation('elu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Dropout(0.2))


    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('elu'))
    model.add(Dropout(0.2))
    model.add(Dense(128))
    model.add(Activation('elu'))
    model.add(Dropout(0.2))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
          #optimizer=SGD(lr=1e-4, momentum=0.9, nesterov=True),
          optimizer=Adam(lr=1e-5),                  
          metrics=['accuracy'])
    return model

def get_net(input_shape, load_weight_path=None, features=False, mal=False):
    inputs = Input(shape=(1, 36, 36, 36), name="input_1")
    x = inputs
    #x = AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), border_mode="same")(x)
    x = Convolution3D(64, 3, 3, 3, activation='relu', border_mode='same', name='conv1', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), border_mode='valid', name='pool1')(x)

    # 2nd layer group
    x = Convolution3D(128, 3, 3, 3, activation='relu', border_mode='same', name='conv2', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool2')(x)
    x = Dropout(p=0.3)(x)

    # 3rd layer group
    x = Convolution3D(256, 3, 3, 3, activation='relu', border_mode='same', name='conv3a', subsample=(1, 1, 1))(x)
    x = Convolution3D(256, 3, 3, 3, activation='relu', border_mode='same', name='conv3b', subsample=(1, 1, 1))(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool3')(x)
    x = Dropout(p=0.4)(x)

    # 4th layer group
    x = Convolution3D(512, 3, 3, 3, activation='relu', border_mode='same', name='conv4a', subsample=(1, 1, 1))(x)
    x = Convolution3D(512, 3, 3, 3, activation='relu', border_mode='same', name='conv4b', subsample=(1, 1, 1),)(x)
    x = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), border_mode='valid', name='pool4')(x)
    x = Dropout(p=0.5)(x)

    last64 = Convolution3D(64, 2, 2, 2, activation="relu", name="last_64")(x)
    out_class = Convolution3D(1, 1, 1, 1, activation="softmax", name="out_class_last")(last64)
    out_class = Flatten(name="out_class")(x)
    
    out_class = Dense(2)(out_class) 
    out_class = Activation('softmax')(out_class)
    
    model = Model(input=inputs, output=out_class)
    model.compile(optimizer=SGD(lr=1e-5, momentum=0.9, nesterov=True), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [3]:
output_path = PATH['cls_train_cube_30']
output_true = PATH['cls_train_cube_30_true']
output_false = PATH['cls_train_cube_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [4]:

def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[0:200]
    file_list_false = get_dirfiles(output_false)[0:200]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)
    
    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,36,36,36])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def valid_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[-40:]
    file_list_false = get_dirfiles(output_false)[-40:]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)

    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,36,36,36])
    labels = np.zeros([nb_true,2])
  
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels


def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):

    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 20, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        #model = model_20()
        #model = preds3d_dense(48)
        #model = classifier((1, 36, 36, 36),64,(3, 3, 3), (2, 2, 2))
        model = get_net((1, 36, 36, 36))
    x,y = train_generator(output_true,output_false)
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs,
              validation_data=valid_generator(output_true,output_false),verbose=1, callbacks=callbacks, shuffle=True)
    
 
    return model

In [5]:
fenlei_fit('Fenge_36_36_36_0624', load_check = False, batch_size=1, epochs=500, check_name = 'Fenge_36_36_36_0615')

100%|██████████| 40/40 [00:00<00:00, 116.28it/s]


Train on 400 samples, validate on 80 samples
Epoch 1/500
400/400 [==============================] - 31s - loss: 0.8105 - acc: 0.4950 - val_loss: 0.7005 - val_acc: 0.5000
Epoch 2/500
400/400 [==============================] - 28s - loss: 0.7497 - acc: 0.5225 - val_loss: 0.7130 - val_acc: 0.5000
Epoch 3/500
400/400 [==============================] - 28s - loss: 0.6846 - acc: 0.5775 - val_loss: 0.7167 - val_acc: 0.5000
Epoch 4/500
400/400 [==============================] - 29s - loss: 0.6635 - acc: 0.6225 - val_loss: 0.6975 - val_acc: 0.5000
Epoch 5/500
400/400 [==============================] - 29s - loss: 0.6659 - acc: 0.5925 - val_loss: 0.6838 - val_acc: 0.5250
Epoch 6/500
400/400 [==============================] - 27s - loss: 0.6362 - acc: 0.6575 - val_loss: 0.7117 - val_acc: 0.5000
Epoch 7/500
400/400 [==============================] - 29s - loss: 0.6628 - acc: 0.6075 - val_loss: 0.6826 - val_acc: 0.5375
Epoch 8/500
400/400 [==============================] - 27s - loss: 0.6308 - acc:

In [6]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [7]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_36_36_36_0624.h5')

In [8]:
#model_pred.load_weights(model_paths + 'Fenge_0613_2.h5')

In [9]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

0.0


In [10]:
cc

[array([[ 0.66788691,  0.33211312]], dtype=float32),
 array([[ 0.63515604,  0.36484396]], dtype=float32),
 array([[ 0.64645052,  0.35354954]], dtype=float32),
 array([[ 0.65018374,  0.34981632]], dtype=float32),
 array([[ 0.47144994,  0.52855003]], dtype=float32),
 array([[ 0.43054485,  0.56945521]], dtype=float32),
 array([[ 0.50334543,  0.49665454]], dtype=float32),
 array([[ 0.63961124,  0.36038867]], dtype=float32),
 array([[ 0.65257978,  0.34742016]], dtype=float32),
 array([[ 0.68107128,  0.31892875]], dtype=float32),
 array([[ 0.51818526,  0.48181471]], dtype=float32),
 array([[ 0.5628863 ,  0.43711367]], dtype=float32),
 array([[ 0.60967028,  0.39032975]], dtype=float32),
 array([[ 0.58112592,  0.41887406]], dtype=float32),
 array([[ 0.56155467,  0.43844539]], dtype=float32),
 array([[ 0.57876164,  0.42123839]], dtype=float32),
 array([[ 0.59775054,  0.40224949]], dtype=float32),
 array([[ 0.58271188,  0.41728818]], dtype=float32),
 array([[ 0.47515875,  0.52484131]], dtype=flo

In [11]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [12]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

0.0


In [13]:
cc

[array([[ 0.4967939 ,  0.50320613]], dtype=float32),
 array([[ 0.46296898,  0.53703105]], dtype=float32),
 array([[ 0.49532223,  0.50467777]], dtype=float32),
 array([[ 0.46887279,  0.53112727]], dtype=float32),
 array([[ 0.45110592,  0.54889411]], dtype=float32),
 array([[ 0.47747648,  0.52252358]], dtype=float32),
 array([[ 0.46281305,  0.53718692]], dtype=float32),
 array([[ 0.44039047,  0.55960947]], dtype=float32),
 array([[ 0.47558555,  0.52441436]], dtype=float32),
 array([[ 0.44372907,  0.55627084]], dtype=float32),
 array([[ 0.42388678,  0.57611322]], dtype=float32),
 array([[ 0.39751291,  0.60248715]], dtype=float32),
 array([[ 0.39481217,  0.60518783]], dtype=float32),
 array([[ 0.38742626,  0.61257374]], dtype=float32),
 array([[ 0.3505753 ,  0.64942479]], dtype=float32),
 array([[ 0.47483307,  0.52516693]], dtype=float32),
 array([[ 0.47152093,  0.5284791 ]], dtype=float32),
 array([[ 0.46393687,  0.53606313]], dtype=float32),
 array([[ 0.58470744,  0.4152925 ]], dtype=flo

In [14]:
len(file_list_false)

3263